In [1]:
!pip install transformers

In [2]:
import json
import pandas as pd
import sys
import os
import glob
import re
from tqdm import tqdm

sys.path.append("../utils")
import vtt2text
from sentence_segmenter import *

### Subtitle Cleansing and Convertion to .txt

In [3]:
input_path = "../data/00_raw"
output_path = "../data/01_preprocessed"

filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(input_path, "*"))]

In [4]:
for filename in tqdm(filenames, total=len(filenames)):
    with open(os.path.join(input_path, filename)) as file:
        text = file.read()
    
    text = vtt2text.remove_tags(text)
    
    lines = text.splitlines()
    lines = vtt2text.remove_header(lines)
    lines = vtt2text.merge_duplicates(lines)
    lines = list(lines)
    
    lines = vtt2text.merge_short_lines(lines)
    lines = list(lines)
    
    final_text = []
    for line in lines:
        line = line.replace("&nbsp;", '')
        if not "-->" in line:
            final_text.append(line)
    
    final_text = ' '.join(final_text)
    with open(os.path.join(output_path, re.sub(r'.vtt', '.txt', filename)), 'w') as file:
        file.writelines(final_text)

100%|██████████████████████████████████████████| 11/11 [00:00<00:00, 148.20it/s]


### Text Segmentation

In [5]:
input_path = "../data/01_preprocessed"
output_path = "../data/02_segmented"

filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(input_path, "*"))]

In [6]:
segmenter = GptSegmenter()

for filename in tqdm(filenames, total=len(filenames)):
    with open(os.path.join(input_path, filename)) as file:
        text = file.read()

    # Get the segmented sentences. Each list is a phrase
    sentences = segmenter.segment(text)

    df = pd.DataFrame(map(lambda words: ' '.join(words), sentences), columns=["text"])
    df.to_csv(os.path.join(output_path, f"segmented-{re.sub(r'.txt', '.csv', filename)}"), index=False)

/home/guilherme-resende/anaconda3/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:902: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
  0%|                                                    | 0/11 [00:00<?, ?it/s]

100%|███████████████████████████████████████████| 11/11 [14:56<00:00, 81.46s/it]
